<a href="https://colab.research.google.com/github/Voidspaceinuniverse/Works/blob/main/Word2vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Импорт библиотек

In [ ]:
%matplotlib inline
#! pip install pymorphy2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import json
from tqdm import tqdm
from sklearn.metrics import *
import warnings
warnings.filterwarnings("ignore")
import os
from sklearn.model_selection import train_test_split
import gensim.downloader
import nltk
from nltk.stem.snowball import SnowballStemmer
import re
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer

#Загрузка датасета

In [ ]:
os.chdir('/content/drive/MyDrive/Colab Notebooks/NLP Course/')
pd.set_option('display.max_columns', None)
df = pd.read_csv('IMDB Dataset.csv', sep=',')
df

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative


#Подготовка данных

In [ ]:
X = df['review']
Y = df['sentiment']

In [ ]:
# токенизация

import nltk
nltk.download('stopwords')
stop_words = nltk.corpus.stopwords.words('english')
stop_words = stop_words + ['director', 'screenwriter', 'starring', 'movies', 'movie']
word_tokenizer = nltk.WordPunctTokenizer()

import re
regex = re.compile(r'[А-Яа-яA-zёЁ-]+')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
def words_only(text, regex=regex):
    try:
        return " ".join(regex.findall(text)).lower()
    except:
        return ""

def process_data(data):
    texts = []

    for item in tqdm(data):

        text_lower = words_only(item)
        tokens = word_tokenizer.tokenize(text_lower)
        tokens = [word for word in tokens if (word not in stop_words and not word.isnumeric())]
        texts.append(tokens)

    return texts

In [ ]:
texts = process_data(X)

100%|██████████| 50000/50000 [00:27<00:00, 1812.52it/s]


In [ ]:
texts[:2]

In [ ]:
# стемминг

from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")
for i in tqdm(range(len(texts))):
    text_stem = [stemmer.stem(x) for x in texts[i]]
    texts[i] = ' '.join(text_stem)

100%|██████████| 50000/50000 [00:55<00:00, 898.27it/s] 


In [ ]:
texts[:2]

In [ ]:
# кодировка таргета

def label2num(y):
    if y == 'positive':
        return 1
    if y == 'negative':
        return 0

encoded_y = [label2num(y) for y in Y]

#Разбиение на Train/Test

In [ ]:
X = texts
Y = encoded_y
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, stratify = Y, test_size = 0.3, random_state = 42)

#Предобработка текста при помощи tf-idf

In [ ]:
vectorizer = TfidfVectorizer(max_features = 2500, norm = None, max_df=1.00, min_df=1, ngram_range=(1, 1))

train_X = vectorizer.fit_transform(X_train)
test_X  = vectorizer.transform(X_test)

#Обучение модели на tf-idf векторах

In [ ]:
clf = RandomForestClassifier(n_estimators = 1500, max_depth = 10)
clf = clf.fit(train_X, Y_train)
pred = clf.predict(test_X)

print('Accuracy: ', accuracy_score(Y_test, pred))
print('F1: ', f1_score(Y_test, pred, average = 'macro'))

Accuracy:  0.8281333333333334
F1:  0.8277275482530371


#Подбор оптимальной комбинации гиперпараметров с помощью Optune

In [ ]:
! pip install optuna -q
import optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.6/409.6 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.8/226.8 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 6.9 MB/s eta 0:00:00


In [ ]:
def optuna_optimize(trial):
    criterion = trial.suggest_categorical('criterion', ['gini','entropy','log_loss'])
    n_estimators = trial.suggest_int('n_estimators', 1000, 1600, 25)
    max_depth = trial.suggest_int('max_depth', 3, 25, 1)
    min_samples_split = trial.suggest_int('min_samples_split', 20, 100, 5)
    max_leaf_nodes = trial.suggest_int('max_leaf_nodes', 80, 150, 5)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 20, 100, 5)
    max_features = trial.suggest_categorical('max_features', ['sqrt', 'log2', None])
    bootstrap = trial.suggest_categorical('bootstrap', [True, False])

    model = RandomForestClassifier(
                                  criterion=criterion,
                                  n_estimators=n_estimators,
                                  max_depth=max_depth,
                                  min_samples_split=min_samples_split,
                                  max_leaf_nodes=max_leaf_nodes,
                                  min_samples_leaf=min_samples_leaf,
                                  bootstrap=bootstrap,
                                  max_features=max_features
    )
    model.fit(train_X, Y_train)
    score = f1_score(Y_test, model.predict(test_X), average='macro')
    return score

In [ ]:
%%time
study_RandomForestClassifier = optuna.create_study(study_name="RandomForestClassifier Optuna Optimization",  direction="maximize")
study_RandomForestClassifier.optimize(optuna_optimize, n_trials=20)
print("Наилучшие значения гиперпараметров {}".format(study_RandomForestClassifier.best_params))
print("f1_score_macro на обучающем наборе: {:.2f}".format(study_RandomForestClassifier.best_value))

In [ ]:
model = RandomForestClassifier(
                                  criterion='entropy',
                                  n_estimators=1300,
                                  max_depth=25,
                                  min_samples_split=50,
                                  max_leaf_nodes=140,
                                  min_samples_leaf=20,
                                  max_features='log2',
                                  bootstrap=False,
                               )

model.fit(train_X,Y_train)
predictions_val = model.predict(test_X)
print(classification_report(Y_test, predictions_val))
print(confusion_matrix(Y_test, predictions_val))

predictions_train = model.predict(train_X)
print(classification_report(Y_train, predictions_train))
print(confusion_matrix(Y_train, predictions_train))

#Векторизация текста с помощью метода word2vec

In [ ]:
import gensim.downloader
word2vec_eng = gensim.downloader.load('word2vec-google-news-300')
word2vec_eng.save('word2vec-google-news-300.model')

[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [ ]:
from gensim.models import Word2Vec
model = Word2Vec.load('word2vec-google-news-300.model')

UnpicklingError: pickle data was truncated

In [ ]:
X = df['review']
Y = df['sentiment']

# токенизация
nltk.download('stopwords')
stop_words = nltk.corpus.stopwords.words('english')
stop_words = stop_words + ['director', 'screenwriter', 'starring', 'movies', 'movie']
word_tokenizer = nltk.WordPunctTokenizer()

regex = re.compile(r'[А-Яа-яA-zёЁ-]+')
texts = process_data(X)

# стемминг
stemmer = SnowballStemmer("english")
for i in tqdm(range(len(texts))):
    text_stem = [stemmer.stem(x) for x in texts[i]]
    texts[i] = ' '.join(text_stem)

# кодировка таргета
def label2num(y):
    if y == 'positive':
        return 1
    if y == 'negative':
        return 0

encoded_y = [label2num(y) for y in Y]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
100%|██████████| 50000/50000 [00:56<00:00, 887.96it/s]


In [ ]:
X = texts
Y = encoded_y
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, stratify = Y, test_size = 0.3, random_state = 42)

vectorizer = TfidfVectorizer(max_features = 2500, norm = None, max_df=1.00, min_df=1, ngram_range=(1, 1))
vectorizer.fit_transform(X_train)
tfidf_feature_names = vectorizer.get_feature_names_out()

In [ ]:
def vectorize_with_tfidf_weights(sentence, model, tfidf_vectorizer, tfidf_feature_names):
    words = sentence.split()
    vector = np.zeros(model.vector_size)
    for word in words:
        if word in model and word in tfidf_feature_names:
            tfidf_weight = tfidf_vectorizer.idf_[tfidf_vectorizer.vocabulary_[word]]
            vector += model[word] * tfidf_weight
    return vector

In [ ]:
#####tfidf_weight = tfidf_vectorizer.vocabulary_[word]

In [ ]:
train_X = [vectorize_with_tfidf_weights(sentence, word2vec_eng, vectorizer, tfidf_feature_names) for sentence in tqdm(X_train)]
test_X = [vectorize_with_tfidf_weights(sentence, word2vec_eng, vectorizer, tfidf_feature_names) for sentence in tqdm(X_test)]

100%|██████████| 15000/15000 [04:31<00:00, 55.31it/s]


In [ ]:
model = RandomForestClassifier(n_estimators = 1500, max_depth = 10)
model.fit(train_X, Y_train)
pred = model.predict(test_X)

print('Accuracy: ', accuracy_score(Y_test, pred))
print('F1: ', f1_score(Y_test, pred, average = 'macro'))

Accuracy:  0.7420666666666667
F1:  0.7420530459210605


In [ ]:
print(classification_report(Y_test, pred))
print(confusion_matrix(Y_test, pred))

              precision    recall  f1-score   support

           0       0.75      0.73      0.74      7500
           1       0.74      0.75      0.74      7500

    accuracy                           0.74     15000
   macro avg       0.74      0.74      0.74     15000
weighted avg       0.74      0.74      0.74     15000

[[5511 1989]
 [1880 5620]]
